In this notebook, we will learn  how to sol the Unity ML-Agents environment for the second project of the Deep Reinforcement Learning Nanodegree program.

In [1]:
from unityagents import UnityEnvironment
from ddpr import *
import numpy as np
env = UnityEnvironment(file_name='Reacher_Linux/Reacher.x86_64')
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
        

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [2]:
env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
num_agents = len(env_info.agents)
print(num_agents)
states = env_info.vector_observations
action_size = brain.vector_action_space_size
# get the current state (for each agent)
scores = np.zeros(num_agents)
replaybuffer=ReplayBuffer(int(1e6))
agent=[]
for _ in range(num_agents//5):
    a = D4PGAgent(1024,states.shape[1],action_size,51,replaybuffer)
    agent.append(a)
print(len(agent))

20
4


In [3]:
from collections import deque
import time
import matplotlib.pyplot as plt
%matplotlib inline
avg_reward = deque(maxlen=100)
max_step = 100
epoches = 200
scores_all=[]
for epoch in range(epoches):
    env_info = env.reset(train_mode=True)[brain_name]
    states = env_info.vector_observations
    scores = np.zeros(num_agents)
    for agent_no in range(num_agents//5):
        agent[agent_no].reset()
    
    start_time = time.time()
    for i in range(1000):
        actions = []
        for agent_no in range(num_agents//5):
            actions.extend(agent[agent_no].policy(states[agent_no*5:agent_no*5+5,:]))# select an action (for each agent)
        actions = np.array(actions)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        for j in range(num_agents):
            replaybuffer.Add(states[j,:], actions[j], rewards[j], next_states[j,:], dones[j])
        if(i%20==0 and len(replaybuffer)>1024):
            for agent_no in range(num_agents//5):
                #print(agent_no)
                for _ in range(10):
                    agent[agent_no].learn()
                #agent[agent_no].update_lr()
        
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    avg_reward.append(np.mean(scores))
    scores_all.append(np.mean(scores))
    duration = time.time() - start_time
    total_average_score=np.mean(avg_reward)
    mean_score=np.mean(scores)
    min_score=np.min(scores)
    max_score=np.max(scores)
    print('\rEpisode {}\tTotal Average Score: {:.2f}\tMean: {:.2f}\tMin: {:.2f}\tMax: {:.2f}\tDuration: {:.2f}'
              .format(epoch, total_average_score, mean_score, min_score, max_score, duration))
    if total_average_score >= 30.0 and epoch >= 100:
        print('Problem Solved after {} epsisodes!! Total Average score: {:.2f}'.format(epoch, total_average_score))
        torch.save(agent[0].actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent[0].critic_local.state_dict(), 'checkpoint_critic.pth')
        break
    
plt.plot(scores_all)
plt.show()
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/vandit/vandit/RL/projects/Assignment-2/d4pg.py:125: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  actor_loss = -(self.z_torch.view(1,-1) * F.softmax(self.critic_local(state,caction))).sum(-1).mean()


Episode 0	Total Average Score: 0.09	Mean: 0.09	Min: 0.00	Max: 0.44	Duration: 790.07
Episode 1	Total Average Score: 0.20	Mean: 0.32	Min: 0.00	Max: 1.14	Duration: 801.27
Episode 2	Total Average Score: 0.37	Mean: 0.70	Min: 0.00	Max: 1.49	Duration: 801.56
Episode 3	Total Average Score: 0.41	Mean: 0.55	Min: 0.05	Max: 1.28	Duration: 800.11
Episode 4	Total Average Score: 0.41	Mean: 0.41	Min: 0.00	Max: 1.18	Duration: 801.37
Episode 5	Total Average Score: 0.43	Mean: 0.54	Min: 0.00	Max: 1.38	Duration: 800.17
Episode 6	Total Average Score: 0.47	Mean: 0.69	Min: 0.00	Max: 1.73	Duration: 800.61
Episode 7	Total Average Score: 0.56	Mean: 1.15	Min: 0.15	Max: 3.02	Duration: 800.85
Episode 8	Total Average Score: 0.61	Mean: 1.08	Min: 0.19	Max: 1.95	Duration: 802.00
Episode 9	Total Average Score: 0.66	Mean: 1.06	Min: 0.11	Max: 2.01	Duration: 800.81
Episode 10	Total Average Score: 0.69	Mean: 1.03	Min: 0.52	Max: 1.66	Duration: 801.02
Episode 11	Total Average Score: 0.77	Mean: 1.59	Min: 0.77	Max: 2.94	Duratio

KeyboardInterrupt: 

In [1]:
trained_agent = DDPGAgent(1024,states.shape[1],action_size,replaybuffer)
trained_agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
trained_agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

NameError: name 'DDPGAgent' is not defined

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
trained_agent.actor_local.eval()
while True:
    tstates = torch.from_numpy(states).to(torch.float32)
    actions = trained_agent.actor_local(tstates).detach().numpy() # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
env.close()